In [1]:
!pip install transformers timm flash_attn einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 78.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 12.2 MB/s eta 0:00:00


In [13]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy

model_id = 'microsoft/Florence-2-large'
modelDense = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)


In [16]:
def run_example(task_prompt, text_input=None, image=None):
    # Ensure the image is provided and is a PIL.Image object
    if image is None or not isinstance(image, Image.Image):
        raise ValueError("The 'image' parameter must be a valid PIL.Image object.")

    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    # Prepare inputs
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    # Generate predictions
    generated_ids = modelDense.generate(
        input_ids=inputs["input_ids"].cuda(),
        pixel_values=inputs["pixel_values"].cuda(),
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

    # Post-process and parse the answer
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_answer

# Example usage
# task_prompt = "<DENSE_REGION_CAPTION>"
task_prompt = "<MORE_DETAILED_CAPTION>"
image_path = "/content/womanDancing.jpg"  # Replace with your image path

# Load the image
image = Image.open(image_path)

# Run the example
answer = run_example(task_prompt=task_prompt, image=image)
print(answer)

{'<MORE_DETAILED_CAPTION>': 'The image shows a young woman in a dance studio. She is wearing a black sports bra and black shorts and is in the middle of a dance move. Her arms are stretched out to the sides and her legs are bent at the knees. The studio has wooden walls and large windows, allowing natural light to enter. The floor is made of light-colored wood. The woman appears to be in the process of performing a dance routine.'}


In [18]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# Load BLIP processor and model
model_id = "Salesforce/blip-image-captioning-base"  # You can also try `blip-large` for better performance
processor = BlipProcessor.from_pretrained(model_id)
modelBlip = BlipForConditionalGeneration.from_pretrained(model_id).eval()


In [19]:
def generate_caption(image):
    # Ensure the image is a valid PIL.Image object
    if image is None or not isinstance(image, Image.Image):
        raise ValueError("The 'image' parameter must be a valid PIL.Image object.")

    # Prepare inputs for the model (only pixel_values is required)
    inputs = processor(images=image, return_tensors="pt")

    # Generate caption
    generated_ids = modelBlip.generate(
        pixel_values=inputs["pixel_values"],  # Use only pixel_values
        max_length=50,  # Adjust based on desired caption length
        num_beams=5,    # Beam search for better quality
    )
    caption = processor.decode(generated_ids[0], skip_special_tokens=True)
    return caption

# Example usage
image_path = "/content/womanDancing.jpg"  # Replace with your image path
image = Image.open(image_path)

caption = generate_caption(image)
print("Generated Image Caption:", caption)

Generated Image Caption: a woman in a black leor leor leor leor leor leor leor leor leor leor leor leo
